In [39]:
import os
from dotenv import load_dotenv


from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain.agents import create_agent


In [32]:
pwd = %pwd
env_path = os.path.join(pwd,'.env')

load_dotenv(env_path)

os.getenv('OPENAI_API_KEY') # Used azure key
model = init_chat_model("gpt-4.1")

In [33]:
model

ChatOpenAI(profile={'max_input_tokens': 1047576, 'max_output_tokens': 32768, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x7f130c66b290>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f130c6cac50>, root_client=<openai.OpenAI object at 0x7f130c775950>, root_async_client=<openai.AsyncOpenAI object at 0x7f130c6c9490>, model_name='gpt-4.1', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [40]:
@tool
def create_calendar_event(
    title:str,
    start_time:str,
    end_time:str,
    atendees:list[str],
    location:str=""
) -> str:
    """create calendar event"""
    return f"Created event: {title} from {start_time} to {end_time} with {len(atendees)} atendees."

@tool
def send_email(
    to:list[str],
    subject:str,
    body:str,
    cc:list[str]
) -> str:
    """send an email via api"""
    return f"Email sent to {','.join(to)} - Subject: {subject}"


@tool
def get_available_time_slots(
    atendees:list[str],
    date:str,
    duration_minutes:int,
)-> list[str]:
    """Check calendar availability for a given atendee on a specific date"""
    return ["09:00","14:00","16:00"]

In [41]:
CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (eg, 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response. "
)



calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slot],
    system_prompt=CALENDAR_AGENT_PROMPT
)